In [1]:
# Ref: https://qiita.com/noca/items/00646efd9d4a7302f522
from ipywidgets import Textarea
import io
if 'open' in globals():
    del open
if 'input' in globals():
    del input
original_open = open
class custom_open():
    def __init__(self):
        self.text = ''
    def __call__(self, file, *args, **kwargs):
        if file == 0:
            return io.StringIO(self.text)
        return original_open(file, *args, **kwargs)
    def updater(self, change):
        self.text = change["new"]
class custom_input():
    def __init__(self):
        self.__sio = io.StringIO('')
        self.shell = get_ipython()
        if self.shell.events.callbacks['pre_run_cell'] != []:
            self.shell.events.callbacks['pre_run_cell'] = []
        self.shell.events.register('pre_run_cell', self.pre_run_cell)
    def __call__(self):
        return self.__sio.readline().strip()
    def pre_run_cell(self, info):
        text = self.shell.user_ns.get('text_area', None).value
        self.__sio = io.StringIO(text)
open = custom_open()
input = custom_input()
text_area = Textarea()
text_area.observe(open.updater, names='value')
display(text_area)

Textarea(value='')

In [9]:
import sys
sys.setrecursionlimit(10 ** 6)
from functools import lru_cache
import bisect

N, M, K = map(int, input().split())

G = [list() for _ in range(N)]
GS = [list() for _ in range(N)]
for _ in range(M):
    A, S, B, T = map(int, input().split())
    A -= 1
    B -= 1
    T += K
    G[A].append((S, T, B))
    GS[A].append(S)

for i in range(N):
    G[i].sort()
    GS[i].sort()

@lru_cache(maxsize = 10 ** 5)
def dfs(pos, time, idx):
    temp = 0
    if idx + 1 < len(G[pos]):
        temp = max(temp, dfs(pos, G[pos][idx+1][0], idx+1))
    
    s, t, nex = G[pos][idx]
    idx_nex = bisect.bisect_left(GS[nex], t)
    if idx_nex == len(G[nex]):
        temp = max(temp, 1)
        return temp
    
    temp = max(temp, dfs(nex, G[nex][idx_nex], idx_nex) + 1)
    return temp

ans = 0
for i in range(N):
    if G[i]:
        s, t, nex = G[i][0]
        ans = max(ans, dfs(i, s, 0))
print(ans)

3
